# adding an http attribrute to pandas series

i really like fluent programming styles which is why i love using pandas objects as first class citizens in my programs.
i've been adding accessors to pandas over time to improve my practice. they do things like jinja templates, subprocess calls,
and awaitables.

a feature i have not added is the ability to call http methods from a series of urls.
also, an appropriately schematized dataframe could support more fine tuning.
we ignore the dataframe case in this notebook and work off the series case.|

In [1]:
    import midgy
    from nobook.utils import *
    from nobook.utils import Accessor, SERIES, INDEX, loads
    from functools import partial

use `aiohttpx` as an async aware http requests tool with a caching client for edge buzz buzz.

In [3]:
    class _http(Accessor, types=SERIES | INDEX, name="http"):
        def __init__(self, object):
            import aiohttp, aiohttp_client_cache
            super().__init__(object)
            self._client = aiohttp.ClientSession
            self._client_cache = partial(
                aiohttp_client_cache.CachedSession, cache=aiohttp_client_cache.SQLiteBackend(".pandas.cache.sqlite")
            )

        def client(self, cache=True):
            if cache:
                return self._client_cache()
            return self._client()
            
        async def get(self, mime=None, cache=True):
            async with self.client(cache) as session:
                data = await apply(self.object, self._get_one, mime=mime, session=session)
            return data

        async def _get_one(self, url, mime=None, session=None):
            async with session.get(url) as response:
                if mime is None:
                    mime = response.headers.get("content-type", "text/plain")
                    mime, *_ = mime.partition(";")
                data = loads(await response.text(), mime)
            return data

example grabbing data about my gist history.

In [6]:
    gists = (
        await (
            "https://api.github.com/users/tonyfast/gists" + Index(range(1, 5)).map("?page={}".format)
        ).http.get("application/json")
    ).explode().series().set_index("id", append=True)
    (files := gists.files.series().stack().series())

filename  \
                                                   id                                                                                                                              
https://api.github.com/users/tonyfast/gists?page=1 90c41d4994f75c594db804aeba56fc26 first_and_second_laws_of_thermodynamics.ipynb  first_and_second_laws_of_thermodynamics.ipynb   
                                                   aa3b16c5a284150e3d727a843b6cefec axe_types.py                                                                    axe_types.py   
                                                   713ae6c57602c0f85d011421b20d5ea0 BinaryExamples.ipynb                                                    BinaryExamples.ipynb   
                                                   c004044b4fe641735031ecf2069cf595 aom.json                                                                            aom.json   
                                                   e17946facd998a931527467d646cc822 README.md                                                                          README.md   
...                                                                                                                                                                          ...   
https://api.github.com/users/tonyfast/gists?page=4 cfb55f41f5452ef33ec6fbb4e0bda991 doctest-myst.ipynb                                                        doctest-myst.ipynb   
                                                   357b09758b5fb0f31fd2c0e7f7cf3967 readme.ipynb                                                                    readme.ipynb   
                                                   e0468f0decb0a454d8fae4de8511d794 df-df-api.ipynb                                                              df-df-api.ipynb   
                                                   292377ff60690bff0bf866e93a36eaf2 reprish.ipynb                                                                  reprish.ipynb   
                                                   c72d948631f3d38702d2876942ceb57e custom_repr.ipynb                                                          custom_repr.ipynb   

                                                                                                                                                   type  \
                                                   id                                                                                                     
https://api.github.com/users/tonyfast/gists?page=1 90c41d4994f75c594db804aeba56fc26 first_and_second_laws_of_thermodynamics.ipynb            text/plain   
                                                   aa3b16c5a284150e3d727a843b6cefec axe_types.py                                   application/x-python   
                                                   713ae6c57602c0f85d011421b20d5ea0 BinaryExamples.ipynb                                     text/plain   
                                                   c004044b4fe641735031ecf2069cf595 aom.json                                           application/json   
                                                   e17946facd998a931527467d646cc822 README.md                                             text/markdown   
...                                                                                                                                                 ...   
https://api.github.com/users/tonyfast/gists?page=4 cfb55f41f5452ef33ec6fbb4e0bda991 doctest-myst.ipynb                                       text/plain   
                                                   357b09758b5fb0f31fd2c0e7f7cf3967 readme.ipynb                                             text/plain   
                                                   e0468f0decb0a454d8fae4de8511d794 df-df-api.ipynb                                          text/plain   
                                                   292377ff60690bff0bf866e93a36eaf2 reprish.ipynb                         

## conclusion

* this is the densest i've ever writtent his practice. i love it.
* `aiohttp` includes a server that could also be attached to the dataframe to serve it; imagine `df.serve()`.